# Comparison between products by plotting spectrum interactively

#### release date: 15/08/2019

## Imports

In [11]:
%matplotlib notebook

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display

from datetime import datetime
import warnings
from IPython.display import clear_output

import logging as log
from utilities.files import fl_start_log
from utilities.util import *

from datacube import Datacube
from os.path import join as pjoin
import os

plotly.offline.init_notebook_mode()

# Step 1: Load relevant data from Datacube

## Functions of setting up the GUI for data loading

def set_database_choices_against_config(sender):
    config_file = config_input.value
    try:
        with open(config_file) as f:
            content = f.readlines()
        content = [x.strip() for x in content] 
        database_choices.options = content 
        database_choices.value = content[0]
    except FileNotFoundError:
        print("File not found. Check the path variable and filename")
        exit()    

def set_product_choices_against_database(change):
    database_choices.value = change['new']
    if '.csv' not in database_choices.value:
        if database_choices.value == 'default':
            ARDremotedc = Datacube()
        else:
            ARDremotedc = Datacube(config=database_choices.value)
        db_listed_products = list(ARDremotedc.list_products().name)
        list_products.options = db_listed_products
        list_products.value = db_listed_products[0]
    else:            
        list_products.options = ['field_data']
        list_products.value = list_products.options[0]

def set_band_choices_against_product(change):
    list_products.value = change['new']
    
    if database_choices.value == 'default':
        ARDremotedc = Datacube()
    else:
        ARDremotedc = Datacube(config=database_choices.value)
    
    meas = ARDremotedc.list_measurements()
    list_bands.options = list(meas.loc[meas.index.get_level_values(0) == list_products.value].index.get_level_values(1))
    list_bands.value = [list_bands.options[0]]
    mask_band.options = list(meas.loc[meas.index.get_level_values(0) == list_products.value].index.get_level_values(1))
    mask_band.value = mask_band.options[0]

def get_band_values(change):
    list_bands.value = change['new']

def get_mask_band_value(change):
    mask_band.value = change['new']

def if_remove_partial(change):
    remove_partial_scenes.value = change['new']

def if_common_dates(change):
    choose_common_dates.value = change['new']

# Step 2 Retrive relevant bands from loaded data and plot

## Functions to set up a GUI, define plotting options and plot the statistical value

def retrieve_products_against_loctime(change):
    available_loc_time.value = change['new']
    orig_loc_time = convert2original_loc_time(available_loc_time.value)    
    retrieved_products = sorted(list(set([list(prod.keys())[0] for prod in loaded_data_list[orig_loc_time] if prod])))
    if len(retrieved_products) > 0:
        available_products.options = retrieved_products
        available_products.value = retrieved_products[0]  

def retrieve_bands_against_product(change):
    available_products.value = change['new']
    orig_loc_time = convert2original_loc_time(available_loc_time.value)
    retrieved_products = [prd for prd in loaded_data_list[orig_loc_time] if prd]
    bands = []
    # add all bands that under the same product name
    for a_prod in retrieved_products:
        if (list(a_prod.keys())[0]) == available_products.value:
            plotting_bands_list = list(a_prod[available_products.value]['data'].data_vars)
            plotting_bands_list.remove(a_prod[available_products.value]['mask_band'])
            bands.extend(plotting_bands_list)
  
    available_bands.options = sorted(list(set(bands)))
    available_bands.value = [bands[0]]

def retrieve_band_values(change):
    available_bands.value = change['new']    

## Loading data via a GUI

def on_add_prod_button_clicked(b):
    clear_output(wait=True)
    display(form) 

    datacube_config = database_choices.value
    source_prod = list_products.value
    source_band_list_no_mask = list_bands.value
    the_mask_band = mask_band.value
    #to retrive the mask band together with the data
    source_band_list = source_band_list_no_mask + (the_mask_band, )    
    
    log.info('Database source {}: {}'.format(str(len(added_product)+1), config_input.value)) 
    log.info('Database choice {}: {}'.format(str(len(added_product)+1), datacube_config))
    log.info('Product {}: {}'.format(str(len(added_product)+1), source_prod))
    log.info('Available bands: {}'.format(list_bands.options))
    log.info('Selected bands to load for plotting: {}'.format(source_band_list_no_mask))
    log.info('Selected mask band: {}'.format(the_mask_band))
                
    if len(added_product) > 0:
        exists = 0
        for a_prod in added_product:
            # if the same product already exists, don't add any more
            if (datacube_config, source_prod, set(source_band_list), the_mask_band) == (a_prod[0], a_prod[1], set(a_prod[2]), a_prod[3]):
                exists = 1
                break
        if exists == 0:
            added_product.append((datacube_config, source_prod, source_band_list, the_mask_band))
    else:
        added_product.append((datacube_config, source_prod, source_band_list, the_mask_band))
    
    if database_choices.value == 'default':
        ARDremotedc = Datacube()
    else:
        ARDremotedc = Datacube(config=database_choices.value)    
    meas = ARDremotedc.list_measurements()    
    all_bands_list =  list(meas.loc[meas.index.get_level_values(0) == source_prod].index.get_level_values(1))
    
    if len(added_product_reports) > 0:
        exist = 0
        for a_prod in added_product_reports:
            # if the same product already exists, don't add any more
            if (datacube_config, source_prod) == (a_prod[0], a_prod[1]):
                exist = 1
                break
        if exist == 0:
            added_product_reports.append((datacube_config, source_prod, all_bands_list, the_mask_band))
    else:
        added_product_reports.append((datacube_config, source_prod, all_bands_list, the_mask_band))        

def load_data(products_to_load, output_data_list):
    
    if len(products_to_load) > 0:        
        acq_min=acq_min_input.value
        acq_max=acq_max_input.value
        log.info('Date range and spatial information ...')
        log.info('Start date: {}'.format(acq_min))
        log.info('End date: {}'.format(acq_max))
        
        no_partial_scenes = remove_partial_scenes.value
        common_dates = choose_common_dates.value
        log.info('No partial secens: {}'.format(no_partial_scenes))
        log.info('Common dates: {}'.format(common_dates))

        loc_method = loc_input.selected_index         
        if loc_method == 0:        
            lon=lon_input.value
            lat=lat_input.value
            window_size = window_size_input.value
            log.info('Single point method ...')
            log.info('Longitude: {}'.format(lon))
            log.info('Latitude: {}'.format(lat))
            log.info('Extent width/height: {} metre'.format(window_size))
           
            for a_prod in products_to_load: 
                # check if the same product has been loaded already
                kwargs = {'loc': (lon, lat)}
                if not aleady_loaded(a_prod, acq_min, acq_max, output_data_list, **kwargs):
                    returned = single_loc_process(a_prod, acq_min, acq_max, lon, lat, window_size, no_partial_scenes)
                    if returned: 
                        if ((lon, lat), (acq_min, acq_max)) not in output_data_list:
                            output_data_list[((lon, lat), (acq_min, acq_max))] = []
                        output_data_list[((lon, lat), (acq_min, acq_max))].append(returned)
                   
        elif loc_method == 1:        
            lon_lat_file = multiple_lon_lat_input.children[0].value
            window_size = window_size_input.value 
            log.info('Multiple points method ...')
            log.info('File containing points: {}'.format(lon_lat_file))
            log.info('Extent width/height: {}'.format(window_size))
            for a_prod in products_to_load:
                output_data_list = multiple_loc_process(a_prod, acq_min, acq_max, lon_lat_file, 
                                                        window_size, no_partial_scenes, output_data_list)
        elif loc_method == 2:        
            single_shape_file = single_polygon_input.value
            log.info('Single polygon method ...')
            log.info('Single polgon shape file: {}'.format(single_shape_file))
           
            for a_prod in products_to_load:
                kwargs = {'loc': single_shape_file}
                if not aleady_loaded(a_prod, acq_min, acq_max, output_data_list, **kwargs):
                    returned = single_shape_process(a_prod, acq_min, acq_max, single_shape_file, no_partial_scenes)
                    if returned:                                   
                        if (single_shape_file, (acq_min, acq_max)) not in output_data_list:
                            output_data_list[(single_shape_file, (acq_min, acq_max))] = []
                        output_data_list[(single_shape_file, (acq_min, acq_max))].append(returned)
        else:        
            multi_shape_file = multiple_polygon_input.value
            log.info('Multiple polygons method ...')
            log.info('File containing multiple shape files: {}'.format(multi_shape_file))
            for a_prod in products_to_load:
                output_data_list = multi_shape_process(a_prod, acq_min, acq_max, multi_shape_file, 
                                                       no_partial_scenes, output_data_list)
                
        # select common dates among the products
        if common_dates:
            output_data_list = get_common_dates(output_data_list) 
            
        print ('Number of location/time loaded so far: {}'.format(len(output_data_list)))
        log.info('Number of location/time loaded so far: {}'.format(len(output_data_list)))
        
        for key, items_list in output_data_list.items():
            print ('Combination of location and time: {}\r'.format(key))
            log.info('Combination of location and time: {}\r'.format(key))
            if len(items_list) < 1:
                print ('No data available')
                log.info('No data available')
            else:
                for item in items_list:
                    prod_name = list(item.keys())[0]                
                    print ('{}: Number of data: {}\r'.format(prod_name, len(item[prod_name]['data'].time)))
                    log.info('{}: Number of data: {}\r'.format(prod_name, len(item[prod_name]['data'].time)))                        
    else:
        print ('No products added for loading')

def on_extract_button_clicked(b):
    clear_output(wait=True)
    display(form) 
    load_data(added_product, loaded_data_list)

def on_output_button_clicked(b):
    clear_output(wait=True)
    display(form)
    
    log.info('\nTo produce output ...')
    log.info('Step 1: load all bands ...')
    load_data(added_product_reports, loaded_data_list_reports)
    log.info('Step 2: produce output csv files ...')
    produce_reports(report_folder, loaded_data_list_reports, choose_common_dates.value)    

def on_load_setting_button_clicked(b):
    
    existing_settings = Dropdown(
                                 options=[],
                                 disabled=False
                                )
    
    setting_file = pjoin(settings_folder, '{}.cfg'.format(settings_choices.value))
    
    if os.path.exists(setting_file):
        config = configparser.RawConfigParser()
        config.read(setting_file)
        
        config_input.value = config.get('Settings', 'config_input')
        database_choices.options = ast.literal_eval(config.get('Settings', 'database_choices_options'))
        database_choices.value = config.get('Settings', 'database_choices_value')
        list_products.options = ast.literal_eval(config.get('Settings', 'list_products_options'))
        list_products.value = config.get('Settings', 'list_products_value')
        list_bands.options = ast.literal_eval(config.get('Settings', 'list_bands_options'))
        list_bands.value = ast.literal_eval(config.get('Settings', 'list_bands_value'))
        mask_band.options = ast.literal_eval(config.get('Settings', 'mask_band_options'))
        mask_band.value = config.get('Settings', 'mask_band_value')
        remove_partial_scenes.value = config.getboolean('Settings', 'remove_partial_scenes')
        choose_common_dates.value = config.getboolean('Settings', 'choose_common_dates')
        acq_min_input.value = config.get('Settings', 'acq_min_input')
        acq_max_input.value = config.get('Settings', 'acq_max_input')
        lon_input.value = config.get('Settings', 'lon_input')
        lat_input.value = config.get('Settings', 'lat_input')
        loc_file_input.value = config.get('Settings', 'loc_file_input')
        window_size_input.value = config.get('Settings', 'window_size_input')
        single_polygon_input.value = config.get('Settings', 'single_polygon_input')
        multiple_polygon_input.value = config.get('Settings', 'multiple_polygon_input')
        loc_input.selected_index = config.get('Settings', 'loc_input')
        
        added_product.extend(ast.literal_eval(config.get('Settings', 'added_product')))
        added_product_reports.extend(ast.literal_eval(config.get('Settings', 'added_product_reports')))

def on_save_setting_button_clicked(b):
    
    setting_file = pjoin(settings_folder, '{}.cfg'.format(new_setting.value))
    
    if os.path.exists(setting_file):
        os.remove(setting_file)
   
    out_txt = open(setting_file, 'w')
    
    out_txt.write('[Settings]\n')
    out_txt.write('config_input = {}\n'.format(config_input.value))
    out_txt.write('database_choices_options = {}\n'.format(database_choices.options))
    out_txt.write('database_choices_value = {}\n'.format(database_choices.value))
    out_txt.write('list_products_options = {}\n'.format(list_products.options))
    out_txt.write('list_products_value = {}\n'.format(list_products.value))
    out_txt.write('list_bands_options = {}\n'.format(list_bands.options))
    out_txt.write('list_bands_value = {}\n'.format(list_bands.value))
    out_txt.write('mask_band_options = {}\n'.format(mask_band.options))
    out_txt.write('mask_band_value = {}\n'.format(mask_band.value))
    out_txt.write('remove_partial_scenes = {}\n'.format(remove_partial_scenes.value))
    out_txt.write('choose_common_dates = {}\n'.format(choose_common_dates.value))
    out_txt.write('acq_min_input = {}\n'.format(acq_min_input.value))
    out_txt.write('acq_max_input = {}\n'.format(acq_max_input.value))
    out_txt.write('lon_input = {}\n'.format(lon_input.value))
    out_txt.write('lat_input = {}\n'.format(lat_input.value))
    out_txt.write('loc_file_input = {}\n'.format(loc_file_input.value))    
    out_txt.write('window_size_input = {}\n'.format(window_size_input.value))
    out_txt.write('single_polygon_input = {}\n'.format(single_polygon_input.value))
    out_txt.write('multiple_polygon_input = {}\n'.format(multiple_polygon_input.value))            
    out_txt.write('loc_input = {}\n'.format(loc_input.selected_index)) 
    
    out_txt.write('added_product = {}\n'.format(added_product))
    out_txt.write('added_product_reports = {}\n'.format(added_product_reports))
    
    out_txt.close()

# set up log file
log_folder = create_sub_folder('.', 'logs')
log_file = pjoin(log_folder, '{}.log'.format(str(datetime.now())))
log_level = 'INFO'
verbose = True
fl_start_log(log_file, log_level, verbose)
log.info('start ...')

# set up report folder
report_folder = create_sub_folder('.', 'reports')

# set up settings folder
settings_folder = create_sub_folder('.', 'settings')

config_input = Text(value='examples/database_sources.txt')
database_choices = Dropdown(
                            options=[],
                            disabled=False
                           )
list_products = Dropdown(
                         options=[],
                         disabled=False
                        )
list_bands = SelectMultiple(
                            options=[],
                            value=[],
                            disabled=False
                           )
mask_band = Dropdown(
                     options=[],
                     disabled=False
                    )


remove_partial_scenes = ToggleButton(
                                 value=False,
                                 description='Cloud Free',
                                 disabled=False,                                  
                                 button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                 tooltip='Only include acquisitions without masked (cloud/shadow) pixels',
                                 indent=False,
                                 icon=''
                                )

choose_common_dates = ToggleButton(
                        value=False,
                        description='Same Dates',
                        disabled=False,
                        button_style='', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip='Only include same day observations',
                        indent=False,
                        icon=''
                       )

config_input.on_submit(set_database_choices_against_config)
database_choices.observe(set_product_choices_against_database, 'value')
list_products.observe(set_band_choices_against_product, 'value')
list_bands.observe(get_band_values, 'value')
mask_band.observe(get_mask_band_value, 'value')
remove_partial_scenes.observe(if_remove_partial, 'value')
choose_common_dates.observe(if_common_dates, 'value')
extra_filters = HBox([remove_partial_scenes, choose_common_dates])

acq_min_input = Text(value='2018-01-01')
acq_max_input = Text(value='2018-03-31')

lon_input = FloatText(value=142.9382, description='Longitude')
lat_input = FloatText(value=-22.5273, description='Latitude')
window_size_input = FloatText(value=90, description='Extent (m)')
single_lon_lat_input = VBox([lon_input, lat_input, window_size_input])

loc_file_input = Text(value='examples/3_points.txt', description='Input file')
multiple_lon_lat_input = VBox([loc_file_input, window_size_input])

single_polygon_input = Text(value='examples/shp/test_plgon.shp', description='Single file')

multiple_polygon_input = Text(value='examples/shape_files.txt', description='Input file')

loc_input = widgets.Tab()
loc_input.children = [single_lon_lat_input, multiple_lon_lat_input, single_polygon_input, multiple_polygon_input]
title = ['Single lon/lat', 'Multiple lon/lat', 'Single polygon', 'Multiple polygons']
for i in range(len(loc_input.children)):
    loc_input.set_title(i, title[i])       

button_add = Button(description='Add Product/Bands', button_style='info', tooltip='Add product with selected bands for extracting')
button_extract = Button(description='Extract Products', button_style='info', tooltip='Extract products with selected bands for plotting')
# button_extract.style.button_color = 'lightblue'
button_output = Button(description='Output Reports', button_style='info', tooltip='Output reports for products with all available bands')
buttons_box = HBox([button_extract, button_output])

button_load_setting = Button(description='Load Settings', button_style='info', tooltip='Load an existing setting')
button_load_setting.style.button_color = 'tan'
settings_choices = Dropdown(
                            options=[os.path.splitext(f)[0] for f in os.listdir(settings_folder)],
                            disabled=False
                           )

button_save_setting = Button(description='Save Settings', button_style='info', tooltip='Save current setting')
button_save_setting.style.button_color = 'tan'
new_setting = Text(value='', tooltip='Please type a name')
buttons_box_setting = HBox([button_load_setting, settings_choices, button_save_setting, new_setting])

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    width='78%'
)

form_items = [
    Box([Label(value='Database source'), config_input], layout=form_item_layout),
    Box([Label(value='Database choice'), database_choices], layout=form_item_layout),
    Box([Label(value='Product'), list_products], layout=form_item_layout),
    Box([Label(value='Bands to load'), list_bands], layout=form_item_layout),
    Box([Label(value='Mask band'), mask_band], layout=form_item_layout), 
    Box([button_add], layout=Layout(justify_content='center')),
    Box([Label(value='Start Date'), acq_min_input], layout=form_item_layout),
    Box([Label(value='End Date'), acq_max_input], layout=form_item_layout),
    Box([Label(value='Spatial location'), loc_input], layout=Layout(justify_content='space-between', width='97%')),
    Box([Label(value='Extra filters'), extra_filters], layout=Layout(justify_content='space-between', width='78%')), 
#     Box([button_extract], layout=Layout(justify_content='center'))
    Box([buttons_box], layout=Layout(justify_content='center')),
#     Box([Label(value='Saved settings'), buttons_box_setting], layout=Layout(justify_content='space-between', width='100%')),
    Box([buttons_box_setting], layout=Layout(justify_content='space-between', width='100%')),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 3px',
    align_items='stretch',
    width='71%'
))                      

display(form)

# added propucts for plotting to load
added_product = []
# loaded products from datacube for plotting
loaded_data_list = {}

# added propucts for reporting to load
added_product_reports = []
# loaded products from datacube for reporting
loaded_data_list_reports = {}


with warnings.catch_warnings():
    warnings.simplefilter("ignore")  
            
    button_add.on_click(on_add_prod_button_clicked)
    button_extract.on_click(on_extract_button_clicked)
    button_output.on_click(on_output_button_clicked)
    button_load_setting.on_click(on_load_setting_button_clicked)
    button_save_setting.on_click(on_save_setting_button_clicked)

Box(children=(Box(children=(Label(value='Database source'), Text(value='examples/database_sources.txt')), layo…

Number of location/time loaded so far: 1
Combination of location and time: ((142.9382, -22.5273), ('2018-01-01', '2018-03-31'))
ls8_ard: Number of data: 6
ls8_usgs_l2c1: Number of data: 6
ls8_ard: Number of data: 6


## Define the plotting options and plot via a GUI 

In [12]:
def on_extract_band_button_clicked(b):
    clear_output(wait=True)
    display(form) 
    if available_loc_time.value:
        loc_time = available_loc_time.value
        prod = available_products.value
        bands = available_bands.value
        
        log.info('Combination of location and time: {}'.format(loc_time))
        log.info('Product: {}'.format(prod))
        print ('Product: {}'.format(prod))
        log.info('Selected bands for plotting: {}'.format(bands))
        print ('Selected bands for plotting: {}'.format(bands))         

        loc_time_orig = convert2original_loc_time(loc_time)
        products_data_list = [prd for prd in loaded_data_list[loc_time_orig] if prd]
        
        for a_band in bands:
            for a_prod_data in products_data_list:
                if list(a_prod_data.keys())[0] == prod:
                    if a_band in list(a_prod_data[prod]['data'].data_vars):
                        loaded_prod_band_list[loc_time_orig][prod]['bands_data'].update({a_band: a_prod_data[prod]['data'][a_band]})
                        loaded_prod_band_list[loc_time_orig][prod]['find_list'] = a_prod_data[prod]['find_list']
                        break
    else:
        print ('Select a location and time first')

def on_plot_button_clicked(b):
    clear_output(wait=True)
    display(form)        

    if available_loc_time.value:    
        loc_time = available_loc_time.value        
        loc_time_orig = convert2original_loc_time(loc_time)
        prod_data = loaded_prod_band_list[loc_time_orig]

        i = 0
        plot_info = {}
        plot_bands = {}
        for a_prod, band_data in prod_data.items():
            if band_data['bands_data']:
                band_choices = []
                for band in list(band_data['bands_data'].keys()):
                    band_choices.append(band)
                band_list = Dropdown(options=band_choices, description=a_prod, layout=band_layout)

                plot_info.update({a_prod: {'data':band_data['bands_data'], 'colour': available_color[i]},})
                plot_bands.update({a_prod: band_list})
                i += 1

        if plot_bands:
            kwargs = plot_bands
            log.info('Plot bands info: {}'.format(plot_bands))
            
            min_reflect = Text(value='', description='Min value')
            max_reflect = Text(value='', description='Max value') 

            i_draw_stat = interactive(draw_stat, plot_info=fixed(plot_info), min_reflect=min_reflect, 
                                      max_reflect=max_reflect, label=fixed(loc_time), **kwargs)
            display(i_draw_stat)
        else:
            print ('No bands selected for plotting')
    else:
        print ('Select a location and time first')

log.info('\n')
log.info('Select and plot bands')

if loaded_data_list:
    loc_time_list = ['{} and {}'.format(a_loc_time[0], a_loc_time[1]) 
                     for a_loc_time in list(loaded_data_list.keys())
                     if loaded_data_list[a_loc_time] != []]
    log.info('Available combinations of spatial location and time: {}'.format(loc_time_list))

    available_loc_time = Dropdown(
                                  options=loc_time_list,
                                  value=None,
                                  disabled=False
                                 )
    available_products = Dropdown(
                                  options=[],
                                  disabled=False
                                 )
    available_bands = SelectMultiple(
                                     options=[],
                                     value=[],
                                     disabled=False
                                    )

    available_loc_time.observe(retrieve_products_against_loctime, 'value')
    available_products.observe(retrieve_bands_against_product, 'value')
    available_bands.observe(retrieve_band_values, 'value')

    button_extract_band_value = Button(description='Add Products/Bands')
    button_extract_band_value.style.button_color = 'lightblue'
    
    button_plot = Button(description='Plot')
    button_plot.style.button_color = 'lightblue'
    
    band_layout = Layout(border='1px black',
                    width='300px',
                    height='30px')

    form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        align_items='stretch',
        width='70%'
    )

    form_items = [
        Box([Label(value='location and time'), available_loc_time], layout=form_item_layout),
        Box([Label(value='products available'), available_products], layout=form_item_layout),
        Box([Label(value='Bands available'), available_bands], layout=form_item_layout),
        Box([button_extract_band_value], layout=Layout(justify_content='center')),
        Box([button_plot], layout=Layout(justify_content='center'))
    ]

    form = Box(form_items, layout=Layout(
        display='flex',
        flex_flow='column',
        border='solid 3px',
        align_items='stretch',
        width='65%'
    ))                      

    display(form) 
    
    available_color = ['blue', 'red', 'orange', 'black', 'green', 'yellow']

    loaded_prod_band_list = {}
    
    for a_loc_time in loc_time_list:
        
        a_loc_time_orig = convert2original_loc_time(a_loc_time)
        loaded_prod_band_list.update({a_loc_time_orig: {}})    

        products = list(set([list(prod.keys())[0] for prod in loaded_data_list[a_loc_time_orig] if prod]))
        if len(products) > 0:
            for prod in products:
                loaded_prod_band_list[a_loc_time_orig].update({
                                                               prod: { 
                                                                      'bands_data': {},
                                                                      'find_list': {}
                                                                      }
                                                               })                                    

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")    

        # extract bands for plotting from previous loaded data
        button_extract_band_value.on_click(on_extract_band_button_clicked)
        button_plot.on_click(on_plot_button_clicked)
else:
    print ('No loaded data from datacube')

Box(children=(Box(children=(Label(value='location and time'), Dropdown(options=("(142.9382, -22.5273) and ('20…

interactive(children=(Text(value='', description='Min value'), Text(value='', description='Max value'), Dropdo…